#### Developed by Senya Stein (senyarocks11@gmail.com) in July 2019 for use with Saildrone data

#### IMMA documentation is at https://rda.ucar.edu/datasets/ds548.0/#!docs

#### Can be used with conversion from IMMA code developed by Zhankun Wang and Phillip Brohan: https://github.com/oldweather/IMMA/blob/master/Python/IMMA/icoads.py

### Note: IMMA expects specific units and instrumentation, this program assumes the following incoming data, instruments, and country codes:

Latitude- Degrees North (hundredths+)

Longitude- Degrees East (hundredths+)

Time- Hours plus minutes

Country that recruited ship- United States (C1_var, will need manual change)

Wind direction compass precision- Tenths+ of degrees

Wind speed units- knots, obtained from anemometer (measured)

Sea level pressure- Hpa

Sea surface temperature- taken with a through hull sensor ASK IF CORRECT TO IMMA PPL (pumped CTD measurment)

Wave period and wave height- don't exist at default, if data includes this, change WD_var, WH_var, WP_var (around line 70)





In [14]:
import numpy as np
import pandas as pn
import xarray as xr
import matplotlib.pyplot as plt
import datetime as dt
import netCDF4

In [15]:
#url = 'https://podaac-opendap.jpl.nasa.gov/opendap/hyrax/allData/insitu/L2/saildrone/Baja/saildrone-gen_4-baja_2018-sd1002-20180411T180000-20180611T055959-1_minutes-v1.nc'
url = './../../data/saildrone-gen_4-baja_2018-sd1002-20180411T180000-20180611T055959-1_minutes-v1.nc'

#change url variable, input your data opendap 
ds = xr.open_dataset(url, drop_variables = {'WING_ANGLE','BARO_PRES_STDDEV', 'ROLL', 'PITCH', 'TEMP_AIR_STDDEV', 'RH_STDDEV', 'UWND_STDDEV', 'VWND_STDDEV', 'GUST_WND_STDDEV', 'TEMP_CTD_STDDEV', 'COND_STDDEV', 'SAL_STDDEV', 'O2_CONC_UNCOR_MEAN', 'O2_CONC_UNCOR_STDDEV', 'O2_SAT_MEAN', 'O2_SAT_STDDEV', 'TEMP_O2_MEAN', 'TEMP_O2_STDDEV', 'CHLOR_MEAN', 'CHLOR_STDDEV', 'BKSCT_RED_MEAN', 'BKSCT_RED_STDDEV', 'CDOM_STDDEV', ' WWND_STDDEV', 'TEMP_IR_UNCOR_STDDEV' })
ds

<xarray.Dataset>
Dimensions:             (obs: 86839, trajectory: 1)
Coordinates:
  * trajectory          (trajectory) float32 1002.0
    time                (trajectory, obs) datetime64[ns] ...
    latitude            (trajectory, obs) float64 ...
    longitude           (trajectory, obs) float64 ...
Dimensions without coordinates: obs
Data variables:
    SOG                 (trajectory, obs) float64 ...
    COG                 (trajectory, obs) float64 ...
    HDG                 (trajectory, obs) float64 ...
    HDG_WING            (trajectory, obs) float64 ...
    BARO_PRES_MEAN      (trajectory, obs) float64 ...
    TEMP_AIR_MEAN       (trajectory, obs) float64 ...
    RH_MEAN             (trajectory, obs) float64 ...
    TEMP_IR_UNCOR_MEAN  (trajectory, obs) float64 ...
    UWND_MEAN           (trajectory, obs) float64 ...
    VWND_MEAN           (trajectory, obs) float64 ...
    WWND_MEAN           (trajectory, obs) float64 ...
    WWND_STDDEV         (trajectory, obs) float64 .

In [16]:
# swap obs for time
ds = ds.isel(trajectory=0)
ds = ds.swap_dims({'obs':'time'})

#pd_ds = ds.to_dataframe()
#dshr = pd_ds.set_index('time').groupby(pd.Grouper(freq='1H')).mean()
#dshr = ds
#resample data to increments of 1 hr
dshr = ds.resample(time='1h', skipna=True, label='left').mean()
dshr.to_netcdf('./../../data/saildrone_resampled.nc')
size = int(len(dshr['time']))
print(size)

1452


In [17]:
dshr = xr.open_dataset('./../../data/saildrone_resampled.nc')
#define variables 
time_shift = dshr.time + np.timedelta64(30,'m')
lat_shift = ds.latitude
lon_shift = ds.longitude
IM = '01'
ATTC = ' '
TI_var = '2'
LI_var = '5'
DSDV = '  '
#NID-2 
NID = '  '
II = '  '#2
ID = '         '#9
C1 = '02'
### C1 is The country that recruited a ship, which may differ from the country of immediate receipt, see page 21, # 16 in IMMA documentation for full list of codes
#VI-1 VV-2 WW-2 W1- 1
VItoW1 = '      '
AtoIT = '    '
WBTItoDPT = '          '


SI_var = '04'
NtoCH = '       '
SDtoHDG = ''#154
ALL_var = ''#458
for i in range(154-1):
    SDtoHDG += ' '
for i in range(458-1):
       ALL_var += ' '
        
for i in range(size-1):
    #format string output and put hours in correct units (.01hr)
    HR = float(time_shift.dt.hour[i]+time_shift.dt.minute[i]/60)
    HR = "{0:.2f}".format(HR).zfill(5)
    HR = str(HR)
           
    time_str = str(time_shift.dt.year[i].data)+str(time_shift.dt.month[i].data).zfill(2)+str(time_shift.dt.day[i].data).zfill(2)+HR
    #time_str is YR + MO + DY + HR
    LAT = float(lat_shift[i])
    LAT = "{0:.2f}".format(LAT).zfill(6)
    LON = float(lon_shift[i])
    LON = "{0:.2f}".format(LON).zfill(7)
    pos_str = str(LAT)+str(LON)

    #DSDV = str(DS_var[i].data)+str(DV_var[i].data)
    
    DI_var = '6'
    D_var = float(np.arctan2(ds.VWND_MEAN[i].data, ds.UWND_MEAN[i].data)*180/np.pi) 
    D_var = "{0:.0f}".format(D_var).zfill(3)
    D_var = str(D_var)
    #this is wind TO direction, is imma wind FROM?
    WI_var = '4'
    W_var = np.sqrt(ds.UWND_MEAN[i].data**2 + ds.VWND_MEAN[i].data**2) #convert vectors to speed
    W_var = float(W_var/.19444)
    W_var = "{0:.1f}".format(W_var).zfill(4) #unit conversion (kn to .1 m/s)
    W_var = str(W_var)
    
    #reformat the SLP and put in correcnt units
    SLP_var = float(ds.BARO_PRES_MEAN[i].data*.1)
    SLP_var = "{0:.1f}".format(SLP_var).zfill(6)
    SLP_var = str(SLP_var)
    IT_var = '8'

    AT_var = ds.TEMP_AIR_MEAN[i]
    AT_var = float(AT_var.data)*.1
    AT_var = "{0:.1f}".format(AT_var).zfill(5)
    AT_var = str(AT_var)
    
    SST_var = ds.TEMP_CTD_MEAN[i]
    SST_var = float(SST_var.data)*.1
    SST_var = "{0:.1f}".format(SST_var).zfill(5)
    SST_var = str(SST_var)
    
    ##IF DS HAS WAVE PERIOD AND HEIGHT DATA, THIS WILL NEED CHANGING
    #some sets have wave data, some don't:
  #  WD_var = '  '
  #  WP_var = str(WAVE_DOMINANT_PERIOD[i].data)
  #  WH_var = str(WAVE_SIGNIFICANT_HEIGHT[i].data)

        #WD-2 WP-2 WH-2
    WD_var = '  '
    WP_var = '  '
    WH_var = '  '
    
    COG_var = ds.COG
    SOG_var = ds.SOG
    
    COG_var = float(COG_var[i].data)
    COG_var = "{0:.0f}".format(COG_var).zfill(3)
    COG_var = str(COG_var)
    
    SOG_var = float((SOG_var[i].data)*1.944)
    SOG_var = "{0:.0f}".format(SOG_var).zfill(2)
    SOG_var = str(SOG_var)
        
    Final_IMMA1_str = time_str + pos_str + IM + ATTC + TI_var + LI_var + DSDV + NID + II + ID + C1 + DI_var + D_var + WI_var + W_var + VItoW1 + SLP_var + AtoIT + IT_var + AT_var + WBTItoDPT + SI_var + SST_var + NtoCH + WD_var + WP_var + WH_var + SDtoHDG + COG_var + SOG_var + ALL_var + '\n' 
    
    f = open("IMMA_saildrone_test.imma", 'a+')
    #name or create your file to write IMMA data to in above line
    f.write(Final_IMMA1_str)
    f.close
    

In [60]:
HR = float(6.2)
#HR = str(HR).format(HR, '%2.2f')#.zfill(5)
#HR = HR.zfill(5)
print(HR)

"{0:.2f}".format(HR).zfill(5)

LAT = float(lat_shift[i])
LAT = "{0:.2f}".format(HR).zfill(6)

print(str(DS_var[1].data))#+str(DV_var[1].data))
ds.UWND_MEAN.units

6.2
286.8000000000002


'm s-1'

In [ ]:
## if data is nan, print('') for it 
## DS amd VS are wrong, what would it be acrually??
#double check W_var calculation